In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import json
from geopandas.tools import sjoin
from sklearn.cluster import KMeans
from shapely.geometry import Point
from geopandas import GeoSeries

In [2]:
HCM_Population_District_Level_shapefile = gpd.read_file('../Data/GIS/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp')
HCM_Population_District_Level_shapefile.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,...,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,...,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,...,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,...,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,...,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,...,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."


In [3]:
data_click_geo = gpd.read_file('../Data/click_data.json')

In [4]:
print(data_click_geo)

       dbh                    geometry
0        1  POINT (106.71136 10.85113)
1        1  POINT (106.71695 10.74449)
2        1  POINT (106.66315 10.83289)
3        1  POINT (106.66267 10.83339)
4        1  POINT (106.65977 10.83729)
...    ...                         ...
83670    1  POINT (106.65664 10.84339)
83671    1  POINT (106.75180 10.85276)
83672    1  POINT (106.73189 10.73154)
83673    1  POINT (106.67640 10.82388)
83674    1  POINT (106.67373 10.82236)

[83675 rows x 2 columns]


In [5]:
HCM_Population_District_Level_shapefile = HCM_Population_District_Level_shapefile.to_crs(epsg=4326)
print(HCM_Population_District_Level_shapefile.crs)

epsg:4326


In [6]:
coordinates = []

In [7]:
with open('../Data/click_data.json') as f:
    data_click = json.load(f)
f.close()

In [8]:
for i in data_click['features']:
    coordinates.append(i['geometry']['coordinates'])


In [9]:
for i in range(10):
    print(coordinates[i][1])

10.851128013
10.7444939572909
10.8328914275087
10.8333854914626
10.8372920629615
10.8553008260429
10.7910760270538
10.8235685751107
10.7491797079915
10.7274324102841


In [34]:
m = folium.Map(tiles='CartoDB positron')

for i in range(10):
    geo_marker = folium.Marker(location = [coordinates[i][1], coordinates[i][0]])
    geo_marker.add_to(m)

m

In [11]:
speed = pd.DataFrame((HCM_Population_District_Level_shapefile["Den_2019"] - HCM_Population_District_Level_shapefile["Den_2017"])/HCM_Population_District_Level_shapefile["Den_2017"])
speed.columns=["Speed"]
newdata = pd.concat([HCM_Population_District_Level_shapefile, speed], axis=1)
# cheese.sort_values(by=['Name of column']).head(5)
top5 = newdata.sort_values(by=["Speed"], ascending=False).head(5)

In [12]:
top5

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,...,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry,Speed
16,District 9,763,District,256257.0,263486.0,271537.0,276432.0,284473.0,284990.0,290620.0,...,2489.70,2494.23,2543.50,2592.32,2719.58,3474.59,53139.220230,1.142649e+08,"POLYGON ((106.84015 10.89893, 106.84048 10.897...",0.277620
23,Hoc Mon,784,District,349065.0,358640.0,366502.0,396763.0,408683.0,414795.0,422471.0,...,3743.18,3799.16,3869.46,3977.58,4085.48,4966.47,65955.793966,1.091744e+08,"POLYGON ((106.58361 10.92869, 106.58391 10.928...",0.215639
18,Nha Be,786,District,101074.0,103793.0,110958.0,119416.0,126062.0,132034.0,139225.0,...,1254.48,1313.90,1385.46,1552.41,1771.22,2058.29,67540.905029,1.004901e+08,"POLYGON ((106.72315 10.72162, 106.72428 10.721...",0.162075
22,District 12,761,District,405360.0,427083.0,455881.0,475824.0,489511.0,499569.0,510326.0,...,9275.44,9466.02,9669.85,9968.06,10403.90,11750.80,47703.394094,5.277342e+07,"POLYGON ((106.69433 10.89842, 106.69446 10.897...",0.129461
17,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,...,2812.21,2809.25,2947.02,3080.46,3243.16,3609.98,39592.970451,4.993869e+07,"POLYGON ((106.74241 10.80855, 106.74442 10.808...",0.113106


In [13]:
HCM_PDL_with_click = sjoin(data_click_geo, HCM_Population_District_Level_shapefile)

In [14]:
HCM_PDL_with_click

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,...,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area
0,1,POINT (106.71136 10.85113),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,...,10028.2000,10487.800,10630.200,10842.20,11065.00,11245.900,11362.300,12410.9000,52828.501430,4.775581e+07
5,1,POINT (106.75131 10.85530),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,...,10028.2000,10487.800,10630.200,10842.20,11065.00,11245.900,11362.300,12410.9000,52828.501430,4.775581e+07
11,1,POINT (106.75553 10.84865),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,...,10028.2000,10487.800,10630.200,10842.20,11065.00,11245.900,11362.300,12410.9000,52828.501430,4.775581e+07
12,1,POINT (106.75589 10.84450),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,...,10028.2000,10487.800,10630.200,10842.20,11065.00,11245.900,11362.300,12410.9000,52828.501430,4.775581e+07
13,1,POINT (106.75193 10.85237),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,...,10028.2000,10487.800,10630.200,10842.20,11065.00,11245.900,11362.300,12410.9000,52828.501430,4.775581e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25540,1,POINT (106.63954 10.74860),3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,...,35594.2000,36105.700,37261.600,36010.20,36256.90,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06
33392,1,POINT (106.64180 10.74428),3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,...,35594.2000,36105.700,37261.600,36010.20,36256.90,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06
39145,1,POINT (106.64165 10.74436),3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,...,35594.2000,36105.700,37261.600,36010.20,36256.90,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06
83226,1,POINT (106.64188 10.74416),3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,...,35594.2000,36105.700,37261.600,36010.20,36256.90,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06


In [15]:
X_train = []
for i in HCM_PDL_with_click["geometry"]:
    point = []
    point.append(i.centroid.x)
    point.append(i.centroid.y)
    X_train.append(point)
for i in range (5):
    print(X_train[i])

[106.71135521, 10.851128013]
[106.751307366056, 10.8553008260429]
[106.755528228011, 10.8486462586696]
[106.755886877046, 10.8445046355836]
[106.751933805447, 10.852373185839]


In [16]:
print(len(X_train))

83675


In [17]:
model = KMeans(n_clusters=20, random_state=0)

model.fit(X_train)

KMeans(n_clusters=20, random_state=0)

In [18]:
cluster_result = model.labels_

In [19]:
unique, counts = np.unique(model.labels_, return_counts=True)
print(unique)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [20]:
print(counts)

[ 4877  4876  7747  5641 12046  4877  5983  5994   980    65  7291  4778
  3779  2685    50   915  3415   103  4719  2854]


In [21]:
center = model.cluster_centers_

In [22]:
print(center)

[[106.73693083  10.73423648]
 [106.67747733  10.82208826]
 [106.7516491   10.85633177]
 [106.63441712  10.82002035]
 [106.65873502  10.84057347]
 [106.67072111  10.79085828]
 [106.70874169  10.74720831]
 [106.76088772  10.84705815]
 [106.71088344  10.85276581]
 [106.52862042  10.9700123 ]
 [106.68703669  10.81877582]
 [106.72982322  10.74432321]
 [106.74755612  10.84374192]
 [106.73736207  10.71828818]
 [106.63407349  10.7050377 ]
 [106.71165807  10.82042383]
 [106.68110921  10.7924153 ]
 [106.82798912  10.84244229]
 [106.71483668  10.74208559]
 [106.75930529  10.83543036]]


In [23]:
center_dict = {'label': [], 'points': [], 'geometry': []}
for i in range(20):
    center_dict['label'].append(i)
    center_dict['geometry'].append(Point(center[i][0], center[i][1]))
    center_dict['points'].append(counts[i])

center_geo = gpd.GeoDataFrame(center_dict, crs="EPSG:4326")

In [24]:
print(center_geo)

    label  points                    geometry
0       0    4877  POINT (106.73693 10.73424)
1       1    4876  POINT (106.67748 10.82209)
2       2    7747  POINT (106.75165 10.85633)
3       3    5641  POINT (106.63442 10.82002)
4       4   12046  POINT (106.65874 10.84057)
5       5    4877  POINT (106.67072 10.79086)
6       6    5983  POINT (106.70874 10.74721)
7       7    5994  POINT (106.76089 10.84706)
8       8     980  POINT (106.71088 10.85277)
9       9      65  POINT (106.52862 10.97001)
10     10    7291  POINT (106.68704 10.81878)
11     11    4778  POINT (106.72982 10.74432)
12     12    3779  POINT (106.74756 10.84374)
13     13    2685  POINT (106.73736 10.71829)
14     14      50  POINT (106.63407 10.70504)
15     15     915  POINT (106.71166 10.82042)
16     16    3415  POINT (106.68111 10.79242)
17     17     103  POINT (106.82799 10.84244)
18     18    4719  POINT (106.71484 10.74209)
19     19    2854  POINT (106.75931 10.83543)


In [25]:
center_with_dist_top5 = sjoin(center_geo, top5)
center_with_dist = sjoin(center_geo, HCM_Population_District_Level_shapefile)

In [26]:
center_with_dist_top5

,label,points,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,...,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Speed
8,8,980,POINT (106.71088 10.85277),22,District 12,761,District,405360.0,427083.0,455881.0,...,9016.09,9275.44,9466.02,9669.85,9968.06,10403.90,11750.80,47703.394094,5.277342e+07,0.129461
17,17,103,POINT (106.82799 10.84244),16,District 9,763,District,256257.0,263486.0,271537.0,...,2419.33,2489.70,2494.23,2543.50,2592.32,2719.58,3474.59,53139.220230,1.142649e+08,0.277620


In [27]:
center_with_dist

,label,points,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,...,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area
0,0,4877,POINT (106.73693 10.73424),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
6,6,5983,POINT (106.70874 10.74721),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
11,11,4778,POINT (106.72982 10.74432),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
13,13,2685,POINT (106.73736 10.71829),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
18,18,4719,POINT (106.71484 10.74209),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
1,1,4876,POINT (106.67748 10.82209),10,Go Vap,764,District,522690.0,548145.0,566216.0,...,28718.400,29854.000,30642.000,31450.300,32163.800,32768.100,33643.100,34332.20,27550.300914,1.971611e+07
4,4,12046,POINT (106.65874 10.84057),10,Go Vap,764,District,522690.0,548145.0,566216.0,...,28718.400,29854.000,30642.000,31450.300,32163.800,32768.100,33643.100,34332.20,27550.300914,1.971611e+07
10,10,7291,POINT (106.68704 10.81878),10,Go Vap,764,District,522690.0,548145.0,566216.0,...,28718.400,29854.000,30642.000,31450.300,32163.800,32768.100,33643.100,34332.20,27550.300914,1.971611e+07
2,2,7747,POINT (106.75165 10.85633),15,Thu Duc,762,District,442177.0,455899.0,478900.0,...,10028.200,10487.800,10630.200,10842.200,11065.000,11245.900,11362.300,12410.90,52828.501430,4.775581e+07
7,7,5994,POINT (106.76089 10.84706),15,Thu Duc,762,District,442177.0,455899.0,478900.0,...,10028.200,10487.800,10630.200,10842.200,11065.000,11245.900,11362.300,12410.90,52828.501430,4.775581e+07


In [28]:
best_of_each_dist_top5 = center_with_dist_top5.loc[center_with_dist_top5.groupby('Dist_Name').points.agg('idxmax')]

In [29]:
best_of_each_dist = center_with_dist.loc[center_with_dist.groupby('Dist_Name').points.agg('idxmax')]
best_of_each_dist

,label,points,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,...,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area
14,14,50,POINT (106.63407 10.70504),14,Binh Chanh,785,District,420109.0,447291.0,469517.0,...,1856.300,1944.790,2033.120,2180.610,2338.380,2463.200,2581.330,2789.32,129933.645113,2.529152e+08
15,15,915,POINT (106.71166 10.82042),9,Binh Thanh,765,District,457362.0,470054.0,484135.0,...,23293.700,23307.600,23231.100,23372.500,23478.900,23559.400,23605.600,24016.80,29544.712130,2.078389e+07
9,9,65,POINT (106.52862 10.97001),13,Cu Chi,783,District,343155.0,355822.0,365779.0,...,841.402,878.246,883.272,898.778,927.109,945.387,963.033,1062.85,119954.769999,4.346845e+08
8,8,980,POINT (106.71088 10.85277),22,District 12,761,District,405360.0,427083.0,455881.0,...,8638.210,9016.090,9275.440,9466.020,9669.850,9968.060,10403.900,11750.80,47703.394094,5.277342e+07
6,6,5983,POINT (106.70874 10.74721),4,District 7,778,District,244276.0,274828.0,268483.0,...,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07
17,17,103,POINT (106.82799 10.84244),16,District 9,763,District,256257.0,263486.0,271537.0,...,2376.490,2419.330,2489.700,2494.230,2543.500,2592.320,2719.580,3474.59,53139.220230,1.142649e+08
4,4,12046,POINT (106.65874 10.84057),10,Go Vap,764,District,522690.0,548145.0,566216.0,...,28718.400,29854.000,30642.000,31450.300,32163.800,32768.100,33643.100,34332.20,27550.300914,1.971611e+07
5,5,4877,POINT (106.67072 10.79086),7,Phu Nhuan,768,District,174535.0,175175.0,177242.0,...,36246.200,36793.300,36903.500,37387.100,37316.800,37174.300,37482.600,33530.30,12392.134315,4.889837e+06
3,3,5641,POINT (106.63442 10.82002),8,Tan Binh,766,District,421724.0,430436.0,434298.0,...,19357.300,19627.100,19747.900,20012.100,20281.400,20927.500,20979.800,21162.20,23807.894025,2.243513e+07
2,2,7747,POINT (106.75165 10.85633),15,Thu Duc,762,District,442177.0,455899.0,478900.0,...,10028.200,10487.800,10630.200,10842.200,11065.000,11245.900,11362.300,12410.90,52828.501430,4.775581e+07
